In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
from collections import namedtuple
from itertools import product
import numpy as np

from pyqumo.random import Exponential, HyperExponential, Erlang
from pyqumo.cqumo.sim import simulate_tandem

In [35]:
Params = namedtuple('Params', [
    'arrival_avg',
    'arrival_std',
    'service_avg',
    'service_std',
    'num_stations',
    'queue_capacity'
])

In [37]:
ARRIVAL_AVG = np.asarray([10, 20])
ARRIVAL_STD = np.asarray([1, 5, 10])
SERVICE_AVG = np.asarray([2.5, 5])
SERVICE_STD = np.asarray([1, 2.5, 5, 7.5, 10])
NUM_STATIONS = np.asarray([5])
QUEUE_CAPACITY = np.asarray([10])

# Build the parameters grid:
parameters = [
    Params(arrival_avg, arrival_std, service_avg, service_std, num_stations, queue_capacity)
    for (arrival_avg, arrival_std, service_avg, service_std, num_stations, queue_capacity) 
    in product(ARRIVAL_AVG, ARRIVAL_STD, SERVICE_AVG, SERVICE_STD, NUM_STATIONS, QUEUE_CAPACITY)
]
print(f"Defined {len(parameters)} parameters grid points")

Defined 60 parameters grid points


In [38]:
# Run the simulation
results = []  # store (params, ret), where `ret` is an instance of `pyqumo.sim.tandem.Result`
NUM_PACKETS = 100000

# This function returns the most appropriate distribution:
def get_distribution(avg, std):
    cv = std / avg
    if cv == 1:
        return Exponential(avg)
    if cv > 1:
        return HyperExponential.fit(avg, std)
    return Erlang.fit(avg, std)


for params in parameters:
    arrival = get_distribution(params.arrival_avg, params.arrival_std)
    services = [
        get_distribution(params.service_avg, params.service_std)
        for _ in range(params.num_stations)
    ]
    ret = simulate_tandem(arrival, services, params.queue_capacity, NUM_PACKETS)
    results.append((params, ret))


In [39]:
# Build a table:
from tabulate import tabulate

rows = []
for (param, ret) in results:
    rows.append((param.arrival_avg, param.arrival_std, param.service_avg, param.service_std, param.queue_capacity, 
                 param.num_stations, ret.delivery_delays[0].avg, ret.delivery_delays[0].std, ret.delivery_prob[0]))
print(tabulate(rows, headers=(
    'Arr.avg.', 'Arr.std.',
    'Srv.avg.', 'Srv. std.',
    'Queue capacity', 'Num. stations',
    'Delay avg.', 'Delay std.',
    'Delivery P.'
)))

  Arr.avg.    Arr.std.    Srv.avg.    Srv. std.    Queue capacity    Num. stations    Delay avg.    Delay std.    Delivery P.
----------  ----------  ----------  -----------  ----------------  ---------------  ------------  ------------  -------------
        10           1         2.5          1                  10                5     12.0198        2.18141       1
        10           1         2.5          2.5                10                5      1.99511       0.890679      1
        10           1         2.5          5                  10                5     20.3976       15.2676        0.99997
        10           1         2.5          7.5                10                5     33.5042       31.4547        0.987119
        10           1         2.5         10                  10                5     46.1059       48.7794        0.938449
        10           1         5            1                  10                5     25.1002        2.18963       1
        10          